# Project

# load data

In [2]:
import pandas as pd
import os


rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings = ratings[['user_id', 'movie_id', 'rating']]
ratings.head()

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [5]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


제목과 id를 대응시키는 딕셔너리를 생성합니다. Series 객체로 만든 후, to_dict() 메소드를 이용합니다. index로 설정한 열이 key가 됩니다

In [7]:
# mpvie_id 와 title을 연결합니다
id_to_title = pd.Series(movies.title.values,index=movies.movie_id).to_dict()
title_to_id = {v:k for k,v in id_to_title.items()}

평점이 매겨진 서로 다른 영화가 몇 종류인지, 사용자는 몇 명인지 확인해 봅니다

In [8]:
# 유저 수
ratings['user_id'].nunique()

6039

In [9]:
# 영화 수
ratings['movie_id'].nunique()

3628

6000명이 평가한 3600개의 영화에 대한 데이터임을 알 수 있습니다.

### Exploration

평점이 높은 영화 30개를 확인해 봅니다

In [10]:
# 평점이 높은 영화
u = pd.DataFrame(ratings.groupby('movie_id')['count'].mean()).sort_values(by='count', ascending=False)

In [11]:
u

,count
movie_id,
1830,5.0
3607,5.0
3800,5.0
3280,5.0
989,5.0
...,...
821,3.0
3876,3.0
2214,3.0


In [12]:
top30_list = u[:30].index.to_list()

In [13]:
list(map(lambda x: id_to_title[x], top30_list))

['Follow the Bitch (1998)',
 'One Little Indian (1973)',
 'Criminal Lovers (Les Amants Criminels) (1999)',
 'Baby, The (1973)',
 'Schlafes Bruder (Brother of Sleep) (1995)',
 'Identification of a Woman (Identificazione di una donna) (1982)',
 'Lured (1947)',
 'Country Life (1994)',
 'Bittersweet Motel (2000)',
 'Gate of Heavenly Peace, The (1995)',
 'Message to Love: The Isle of Wight Festival (1996)',
 'Paralyzing Fear: The Story of Polio in America, A (1998)',
 'Late Bloomers (1996)',
 'Song of Freedom (1936)',
 'Black Sunday (La Maschera Del Demonio) (1960)',
 'Zachariah (1971)',
 'Foreign Student (1994)',
 'Ulysses (Ulisse) (1954)',
 'Smashing Time (1967)',
 'I Am Cuba (Soy Cuba/Ya Kuba) (1964)',
 'Lamerica (1994)',
 'Seven Chances (1925)',
 'Apple, The (Sib) (1998)',
 'Firelight (1997)',
 'Sunset Strip (2000)',
 'Sanjuro (1962)',
 'Seven Samurai (The Magnificent Seven) (Shichinin no samurai) (1954)',
 'World of Apu, The (Apur Sansar) (1959)',
 'Godfather, The (1972)',
 'Shawshank 

# 내가 선호하는 영화 입력하기

입력하기 위해서는, 영화 제목을 찾아야 할 것 같습니다

In [14]:
def search_movie(title):
    return movies[movies['title'].str.contains(title)]['title']

In [15]:
search_movie('Lion')

360    Lion King, The (1994)
Name: title, dtype: object

In [16]:
search_movie('Star')

122        Star Maker, The (Uomo delle stelle, L') (1995)
129                              Frankie Starlight (1995)
195                   Stars Fell on Henrietta, The (1995)
257             Star Wars: Episode IV - A New Hope (1977)
313                                       Stargate (1994)
325                         Star Trek: Generations (1994)
790                                      Lone Star (1996)
1025                              Unhook the Stars (1996)
1178    Star Wars: Episode V - The Empire Strikes Back...
1192    Star Wars: Episode VI - Return of the Jedi (1983)
1335                      Star Trek: First Contact (1996)
1350                 Star Trek: The Motion Picture (1979)
1351        Star Trek VI: The Undiscovered Country (1991)
1352               Star Trek V: The Final Frontier (1989)
1353                  Star Trek: The Wrath of Khan (1982)
1354           Star Trek III: The Search for Spock (1984)
1355                 Star Trek IV: The Voyage Home (1986)
1387          

In [17]:
search_movie('Earth')

461                                 Heaven & Earth (1993)
1233                Day the Earth Stood Still, The (1951)
1259                                Night on Earth (1991)
1874                   Greatest Show on Earth, The (1952)
2466                                    Earthquake (1974)
2596                  Earth Vs. the Flying Saucers (1956)
2844    Mating Habits of the Earthbound Human, The (1998)
3378                               Good Earth, The (1937)
3384                                 Here on Earth (2000)
3524                             Battlefield Earth (2000)
3849                 Hellraiser III: Hell on Earth (1992)
Name: title, dtype: object

In [18]:
search_movie('Hell')

607             Hellraiser: Bloodline (1996)
1919    Hello Mary Lou: Prom Night II (1987)
2809                       Hell Night (1981)
3074              Hell in the Pacific (1968)
3157           Hellhounds on My Trail (1999)
3847                       Hellraiser (1987)
3848         Hellbound: Hellraiser II (1988)
3849    Hellraiser III: Hell on Earth (1992)
Name: title, dtype: object

In [19]:
my_favorite_title = ['Godfather, The (1972)' , 'Hellraiser: Bloodline (1996)' ,'Good Earth, The (1937)' ,\
               'Star Wars: Episode IV - A New Hope (1977)' ,'Lion King, The (1994)']

my_favorite = list(map(lambda x: title_to_id[x], my_favorite_title))

# 'hjahn'이라는 user_id가 위 영화의 평점을 5점을 주었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['hjahn']*5, 'movie_id': my_favorite, 'count':[5]*5})

if not ratings.isin({'user_id':['hjahn']})['user_id'].any():  # user_id에 'hjahn'이라는 데이터가 없다면
    ratings = ratings.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,count
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4
1000208,6040,1097,4
0,hjahn,858,5
1,hjahn,611,5
2,hjahn,3447,5
3,hjahn,260,5
4,hjahn,364,5


# Data Preprocessing

csr 형태로 만들기 위해 unique 값들을 정수 인코딩하는 처리를 거칩니다

In [20]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_uniq = ratings['user_id'].unique()
movie_uniq = ratings['movie_id'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
usr_to_idx = {v:k for k,v in enumerate(user_uniq)}
movie_to_idx = {v:k for k,v in enumerate(movie_uniq)}

In [21]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.
ratings_copy = ratings.copy()
# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(usr_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_artist_data = ratings['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_artist_data) == len(ratings):
    print('movie_id column indexing OK!!')
    ratings['movie_id'] = temp_artist_data
else:
    print('artist column indexing Fail!!')

ratings

user_id column indexing OK!!
movie_id column indexing OK!!


,user_id,movie_id,count
0,0,0,5
1,0,1,3
2,0,2,3
3,0,3,4
4,0,4,5
...,...,...,...
0,6039,607,5
1,6039,1827,5
2,6039,529,5
3,6039,44,5


In [22]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
print(num_user)
num_movie = ratings['movie_id'].nunique()
print(num_movie)
csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.movie_id)), shape= (num_user, num_movie))
csr_data

6040
3628


<6040x3628 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

# Train  Model

In [23]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [24]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=196, regularization=0.01, use_gpu=False, iterations=45, dtype=np.float32)

In [25]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [26]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/45 [00:00<?, ?it/s]

# 결과 분석 

In [27]:
hjahn, hellraiser = usr_to_idx['hjahn'], movie_to_idx[title_to_id['Hellraiser: Bloodline (1996)']]
hjahn_vector, hellraiser_vector = als_model.user_factors[hjahn], als_model.item_factors[hellraiser]

In [28]:
np.dot(hjahn_vector, hellraiser_vector)

0.17275421

영화 제목을 주고 제 선호도를 맞춰 봅니다

In [29]:
def predict_movie_preferences(movie_list):
    movie_prefs=[]
    for movie in movie_list:
        movie_temp = movie_to_idx[title_to_id[movie]]
        movie_pref = np.dot(hjahn_vector, als_model.item_factors[movie_temp])
        movie_prefs.append((movie, movie_pref))
    return movie_prefs

In [30]:
predict_movie_preferences(my_favorite_title)

[('Godfather, The (1972)', 0.791988),
 ('Hellraiser: Bloodline (1996)', 0.17275421),
 ('Good Earth, The (1937)', 0.14107114),
 ('Star Wars: Episode IV - A New Hope (1977)', 0.72605187),
 ('Lion King, The (1994)', 0.47356385)]

비슷한 영화를 나열해 봅니다

In [31]:
idx_to_movie = {v:k for k,v in movie_to_idx.items()}

In [32]:
def get_similar_movies(movie_title: str):
    movie_idx = movie_to_idx[title_to_id[movie_title]]
    similar_movies = als_model.similar_items(movie_idx)
    similar_movies = [id_to_title[idx_to_movie[i[0]]] for i in similar_movies]
    return similar_movies

print("슝=3")

슝=3


In [33]:
get_similar_movies('Hellraiser: Bloodline (1996)')

['Hellraiser: Bloodline (1996)',
 'Hellbound: Hellraiser II (1988)',
 'Hellraiser III: Hell on Earth (1992)',
 'Candyman: Farewell to the Flesh (1995)',
 'Halloween: The Curse of Michael Myers (1995)',
 'Halloween 5: The Revenge of Michael Myers (1989)',
 'Species II (1998)',
 'Phantasm III: Lord of the Dead (1994)',
 'Graveyard Shift (1990)',
 'Puppet Master 5: The Final Chapter (1994)']

영화를 추천받아 봅니다

In [34]:
movie_recommended = als_model.recommend(hjahn, csr_data, N=20, filter_already_liked_items=True)
[id_to_title[idx_to_movie[i[0]]] for i in movie_recommended]

['Godfather: Part II, The (1974)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Beauty and the Beast (1991)',
 'Aladdin (1992)',
 'Raiders of the Lost Ark (1981)',
 'Star Wars: Episode VI - Return of the Jedi (1983)',
 'Wizard of Oz, The (1939)',
 'Forrest Gump (1994)',
 'Star Wars: Episode I - The Phantom Menace (1999)',
 'Mulan (1998)',
 'Toy Story (1995)',
 'Tarzan (1999)',
 'Antz (1998)',
 'Nightmare Before Christmas, The (1993)',
 'Little Mermaid, The (1989)',
 'Jurassic Park (1993)',
 'Snow White and the Seven Dwarfs (1937)',
 'Game, The (1997)',
 'African Queen, The (1951)',
 'Alien (1979)']

# 회고 

1️⃣ 잘한 점:

미리미리 완성했다.

추천 시스템의 결과가 충분히 의미있다.

2️⃣ 어려웠던 점:

CSR 행렬을 구현할 때, 노드를 베끼지 않고 바로 짜보려고 했으나, 반복적인 오류로 포기했다. 

노드 방식을 따라가면서, movie_id와 title을 한번 딕셔너리로 만들고, 또 다시 movie_idx와 movie_id를 딕셔너리로 만들어, 코드의 가독성이 떨어진다.


3️⃣ 느낀 점:

지금까지 한 Exploration 중 가장 노드를 충실히 따라갔다. 채점 기준 역시 노드에서 진행한 것 위주였다. 그러나 조금이라도 다른 태스크가 주어진다면 성공하기 힘들 것 같다는 마음이 강하게 든 프로젝트였다. 

추천 시스템을 발전시킬 수 있는 방법이 많을 것 같다. 추가적인 자료조사가 필요하다.

### 참고한 것 - 유현지 님의 회고